In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 6.3MB/s 
     |████████████████████████████████| 61kB 22.9MB/s 
     |████████████████████████████████| 471kB 43.7MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536110 sha256=f5992bc197cf4d57d40211882432da8ffd5fce9e3f9c460dc453f24ce8b7948d
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=28bcf8d5025fe554496bad3e4a7e844b046d7b5e8ec4ee605124453e8d142a26
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=743fdfade904030794e644695873a66244d2d0fb765b053b29074e96bb4f007c
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
#include <thrust/sort.h>
#include <thrust/execution_policy.h>
extern "C"{
__global__ void sort_thrust(int num_component, int *arr){
thrust::sort(thrust::device, arr, arr + num_component);
}

__global__ void sort_by_key_thrust(int num_component, int *key, int *arr){
thrust::sort_by_key(thrust::device, key, key + num_component, arr);
}
}
""", no_extern_c=True)

In [ ]:
sort_thrust = module.get_function("sort_thrust")
sort_by_key_thrust = module.get_function("sort_by_key_thrust")

In [ ]:
nx = np.int32(6)
arr = np.arange(nx, dtype=np.int32)

In [ ]:
np.random.shuffle(arr)

In [ ]:
arr

array([3, 5, 4, 2, 0, 1], dtype=int32)

In [ ]:
arr_gpu = gpuarray.to_gpu(arr)
arr_gpu2 = arr_gpu.copy()

In [ ]:
sort_thrust(nx, arr_gpu, block=(1, 1, 1), grid=(1, 1, 1))

In [ ]:
arr_gpu.get()

array([0, 1, 2, 3, 4, 5], dtype=int32)

In [ ]:
key = np.arange(nx, 0, -1, dtype=np.int32)

In [ ]:
key_gpu = gpuarray.to_gpu(key)

In [ ]:
sort_by_key_thrust(nx, key_gpu, arr_gpu2, block=(1, 1, 1), grid=(1, 1, 1))

In [ ]:
arr_gpu2.get()

array([1, 0, 2, 4, 5, 3], dtype=int32)

In [ ]:
arr

array([3, 5, 4, 2, 0, 1], dtype=int32)

In [ ]:
key

array([6, 5, 4, 3, 2, 1], dtype=int32)